In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.2.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:4 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:13 http://security.ubuntu.c

In [6]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-10-21 18:27:21--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  --.-KB/s    in 0.09s   

2022-10-21 18:27:21 (10.7 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [3]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Pet_Products_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Pet_Products_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   28794885| REAKC26P07MDN|B00Q0K9604|     510387886|(8-Pack) EZwhelp ...|    Pet Products|          5|            0|          0|   N|                Y|A great purchase ...|Best belly bands ...| 2015-08-31|
|         US|   11488901|R3NU7OMZ4HQIEG|B00MBW5O9W|     912374672|Warren Eckstein's...|    Pet Products|          2|    

In [4]:
from pyspark.sql.functions import to_date

In [5]:
# Create the vine_table. DataFrame
vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])

vine_df.printSchema()

vine_df.show()

root
 |-- review_id: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| REAKC26P07MDN|          5|            0|          0|   N|                Y|
|R3NU7OMZ4HQIEG|          2|            0|          1|   N|                Y|
|R14QJW3XF8QO1P|          5|            0|          0|   N|                Y|
|R2HB7AX0394ZGY|          5|            0|          0|   N|                Y|
| RGKMPDQGSAHR3|          5|            0|          0|   N|                Y|
|R1DJCVPQGCV66E|          5|            0|          0|   N|                Y|
|R3V52EAWLPBFQG|          3|    

## Step 1

In [7]:
# select all rows except review_id
df_reviews = df.select(["star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
# df_reviews.show()

# filter df_reviews for all rows where total_votes >= 20
df_reviews_20 = df_reviews.filter(df_reviews["total_votes"] >= 20)
df_reviews_20.show(10)

+-----------+-------------+-----------+----+-----------------+
|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+-----------+-------------+-----------+----+-----------------+
|          1|           27|         31|   N|                Y|
|          5|           25|         25|   N|                Y|
|          3|           29|         31|   N|                Y|
|          2|           35|         42|   N|                Y|
|          4|           27|         28|   N|                Y|
|          5|           62|         64|   N|                N|
|          2|           36|         43|   N|                Y|
|          5|           20|         20|   N|                Y|
|          1|           20|         23|   N|                Y|
|          5|           35|         36|   N|                Y|
+-----------+-------------+-----------+----+-----------------+
only showing top 10 rows



## Step 2

In [8]:
# filter df_reviews_20 for all rows where helpful_votes / total_votes >= 0.5
df_reviews_50 = df_reviews_20.filter(df_reviews_20["helpful_votes"]/df_reviews_20["total_votes"] >= 0.50)
df_reviews_50.show(5)

+-----------+-------------+-----------+----+-----------------+
|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+-----------+-------------+-----------+----+-----------------+
|          1|           27|         31|   N|                Y|
|          5|           25|         25|   N|                Y|
|          3|           29|         31|   N|                Y|
|          2|           35|         42|   N|                Y|
|          4|           27|         28|   N|                Y|
+-----------+-------------+-----------+----+-----------------+
only showing top 5 rows



## Step 3

In [9]:
# filter df_reviews_50 for all rows where vine = "Y"
paid_reviews_df = df_reviews_50.filter(df_reviews_50["vine"] == "Y")
paid_reviews_df.show(5)

+-----------+-------------+-----------+----+-----------------+
|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+-----------+-------------+-----------+----+-----------------+
|          2|           27|         30|   Y|                N|
|          5|           72|         72|   Y|                N|
|          5|           39|         42|   Y|                N|
|          5|           29|         30|   Y|                N|
|          2|           20|         20|   Y|                N|
+-----------+-------------+-----------+----+-----------------+
only showing top 5 rows



In [10]:
# explore summary stats using describe()
paid_reviews_df.describe().show()

+-------+------------------+------------------+------------------+----+-----------------+
|summary|       star_rating|     helpful_votes|       total_votes|vine|verified_purchase|
+-------+------------------+------------------+------------------+----+-----------------+
|  count|               170|               170|               170| 170|              170|
|   mean| 3.929411764705882|63.641176470588235| 68.03529411764706|null|             null|
| stddev|1.1124708391270806|107.53304464635703|110.81862649267784|null|             null|
|    min|                 1|                13|                20|   Y|                N|
|    max|                 5|              1239|              1277|   Y|                Y|
+-------+------------------+------------------+------------------+----+-----------------+



## Step 4

In [11]:
# filter df_reviews_50 for all rows where vine = "N"
unpaid_reviews_df = df_reviews_50.filter(df_reviews_50["vine"] == "N")
unpaid_reviews_df.show(5)

+-----------+-------------+-----------+----+-----------------+
|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+-----------+-------------+-----------+----+-----------------+
|          1|           27|         31|   N|                Y|
|          5|           25|         25|   N|                Y|
|          3|           29|         31|   N|                Y|
|          2|           35|         42|   N|                Y|
|          4|           27|         28|   N|                Y|
+-----------+-------------+-----------+----+-----------------+
only showing top 5 rows



In [12]:
# explore summary stats using describe()
unpaid_reviews_df.describe().show()

+-------+------------------+------------------+------------------+-----+-----------------+
|summary|       star_rating|     helpful_votes|       total_votes| vine|verified_purchase|
+-------+------------------+------------------+------------------+-----+-----------------+
|  count|             37840|             37840|             37840|37840|            37840|
|   mean|3.7647463002114163| 57.50052854122622|61.863583509513745| null|             null|
| stddev|1.6055561346315699|107.60541722886974|111.92734632290404| null|             null|
|    min|                 1|                10|                20|    N|                N|
|    max|                 5|              5735|              5811|    N|                Y|
+-------+------------------+------------------+------------------+-----+-----------------+



## Step 5

## Paid reviews

In [13]:
# determine count of paid_5star_reviews
paid_5star_reviews = paid_reviews_df[paid_reviews_df["star_rating"] == 5].count()
paid_5star_reviews

65

In [14]:
# Determine count of paid_reviews
paid_reviews = paid_reviews_df.count()
paid_reviews

170

In [15]:
# determine percentage of paid_5star_reviews
percentage_paid_5star_reviews = paid_5star_reviews / paid_reviews * 100
percentage_paid_5star_reviews

38.23529411764706

## Unpaid reviews

In [16]:
# determine count of unpaid_5star_reviews
unpaid_5star_reviews = unpaid_reviews_df[unpaid_reviews_df["star_rating"] == 5].count()
unpaid_5star_reviews

20612

In [17]:
# Determine count of unpaid_reviews
unpaid_reviews = unpaid_reviews_df.count()
unpaid_reviews

37840

In [18]:
# determine percentage of unpaid_5star_reviews
percentage_unpaid_5star_reviews = unpaid_5star_reviews / unpaid_reviews * 100
percentage_unpaid_5star_reviews

54.471458773784356